In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

In [5]:
import pandas as pd
df = pd.read_csv("C:/datasets/diabetes.csv")

In [9]:
# feature와 labels 분리
features = df.drop('Outcome', axis=1)
labels = df['Outcome']

In [10]:
# 데이터 스케일링 
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

In [11]:
# 훈련데이터, 테스트데이터 분리
X_train, X_test, y_train, y_test = train_test_split(features_scaled, labels, test_size=0.2, random_state=42)

In [12]:
# 데이터를 PyTorch 텐서로 변환
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

In [13]:
# DataLoader 생성
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [14]:
# CNN 모델 정의
class DiabetesCNN(nn.Module):
    def __init__(self):
        super(DiabetesCNN, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=16, kernel_size=2, stride=1)
        self.conv2 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=2, stride=1)
        self.fc1 = nn.Linear(32 * 6, 64)
        self.fc2 = nn.Linear(64, 2)

    def forward(self, x):
        x = x.unsqueeze(1)
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = DiabetesCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [15]:
# 모델 학습
for epoch in range(10):  # 에포크 수 조정 가능
    model.train()
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

In [16]:
# 모델 저장
torch.save(model.state_dict(), 'diabetes_cnn.pth')

In [17]:
# 저장된 모델 불러오기
model = DiabetesCNN()
model.load_state_dict(torch.load('diabetes_cnn.pth'))

C:\Users\yujin\AppData\Local\Temp\ipykernel_8380\2004481485.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('diabetes_cnn.pth'))


<All keys matched successfully>

In [21]:
# 추가적인 층 추가
class FineTunedDiabetesCNN(nn.Module):
    def __init__(self):
        super(FineTunedDiabetesCNN, self).__init__()
        self.base_model = model
        self.fc3 = nn.Linear(2, 1)  # 새로운 출력층

    def forward(self, x):
        x = self.base_model(x)
        x = torch.sigmoid(self.fc3(x))  # 이진 분류를 위한 시그모이드 출력
        return x

fine_tuned_model = FineTunedDiabetesCNN()

In [19]:
# 손실 함수 및 옵티마이저 재정의
criterion = nn.BCELoss()  # 이진 분류 손실 함수
optimizer = optim.Adam(fine_tuned_model.parameters(), lr=0.0001)

In [20]:
# 미세 조정 학습
for epoch in range(5):
    fine_tuned_model.train()
    for inputs, targets in train_loader:
        targets = targets.float().unsqueeze(1)  # BCE 손실을 위해 형태 변경
        optimizer.zero_grad()
        outputs = fine_tuned_model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()